# コース別過去10走着順を検証
- 指定 race_id / player_id
- 「コース別」の扱い：レース前には枠番しか判らないためwakuban推奨
  - 「そのレースでの entry もしくは wakuban 基準」における直前N走（最大N件）の着情報を抽出

In [19]:


import pandas as pd
import numpy as np
import re
from pathlib import Path

# ===== 設定（必要に応じて変更） =====
RAW_DIR = Path("../data/raw")
START_DATE = "2024-04-01"        # 期間下限（inclusive）
END_DATE   = "2025-10-06"        # 期間上限（inclusive）
TARGET_RACE_ID   = "202510062412"  # 検証したい race_id
TARGET_PLAYER_ID = "5217"          # 検証したい選手
N_LAST = 10                        # 直前N走
MODE = "wakuban"                     # "entry" or "wakuban"（基準の切替）

# ===== ヘルパ =====
ZEN2HAN = str.maketrans("０１２３４５６７８９ＦＬ．－", "0123456789FL.-")

def norm_str(x):
    return "" if x is None else str(x).strip()

def normalize_rank_token(s: str) -> str:
    """ rank の全角→半角と軽い正規化 """
    t = norm_str(s).translate(ZEN2HAN)
    return t

def is_started_from_rank_token(tkn: str) -> bool:
    """
    出走判定：数値着は True。空/ダッシュ類は False。
    非数値は「欠」だけ False、その他（F/L/転/落/妨/不/エ/沈 等）は True（分母に含める）。
    """
    if tkn == "" or tkn in {"-", "—", "ー", "―"}:
        return False
    if re.fullmatch(r"\d+", tkn):  # 数値着
        return True
    # 'F.01' / 'L.03' を F/L に潰す
    m = re.match(r"^([FL])(?:\.\d+)?$", tkn, flags=re.I)
    if m:
        tkn = m.group(1).upper()
    # 先頭1文字で判定
    first = tkn[0]
    if first == "欠":
        return False
    return True

def parse_date_series(s: pd.Series) -> pd.Series:
    """ YYYYMMDD or 任意文字列→datetime """
    try:
        return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
    except Exception:
        return pd.to_datetime(s, errors="coerce")

# ===== raw の読み込み（全CSV連結） =====
files = sorted(RAW_DIR.glob("*.csv"))
if not files:
    raise FileNotFoundError(f"No CSVs in {RAW_DIR}")
frames = []
for p in files:
    dfi = pd.read_csv(p, dtype=str, encoding="utf-8-sig", keep_default_na=False, engine="python")
    dfi["__source_file"] = p.name
    frames.append(dfi)
raw = pd.concat(frames, ignore_index=True, sort=False)

# ===== 正規化（除外しない） =====
# 型・キー列
raw["race_id"]   = raw.get("race_id", "").astype(str)
raw["player_id"] = raw.get("player_id", "").astype(str)

# entry / wakuban は Int64（欠損許容）
raw["entry"]   = pd.to_numeric(raw.get("entry", np.nan), errors="coerce").astype("Int64")
raw["wakuban"] = pd.to_numeric(raw.get("wakuban", np.nan), errors="coerce").astype("Int64")

# 日付
raw["date"] = parse_date_series(raw.get("date"))
# 期間フィルタ（inclusive）
start_dt = pd.to_datetime(START_DATE)
end_dt   = pd.to_datetime(END_DATE)
raw = raw[(raw["date"] >= start_dt) & (raw["date"] <= end_dt)].copy()

# rank 正規化と数値化
raw["rank_tok"] = raw.get("rank", "").apply(normalize_rank_token)
raw["rank_num"] = pd.to_numeric(raw["rank_tok"], errors="coerce")  # 数値着のみ数値

# 出走判定（分母用）
raw["started_mask"] = raw["rank_tok"].apply(is_started_from_rank_token)

# ST（必要なら参照）: 'F.01'→-0.01 / 'L.03'→+0.03 の符号付き秒
def parse_st(val):
    if val is None: return np.nan
    t = str(val).strip().translate(ZEN2HAN)
    if t == "" or t in {"-", "—", "ー", "―"}: return np.nan
    m = re.match(r"^\d+\s*([FL](?:\.\d+)?)$", t, flags=re.I)
    if m: t = m.group(1)
    sign = 1.0
    if t[:1].upper() == "F":
        sign; sign = -1.0
        t = t[1:].strip()
    elif t[:1].upper() == "L":
        t = t[1:].strip()
    if re.fullmatch(r"\d{2}", t): t = "0." + t
    if t.startswith("."): t = "0" + t
    if not re.fullmatch(r"\d+(\.\d+)?", t): return np.nan
    try: return sign * float(t)
    except: return np.nan

if "ST" in raw.columns:
    raw["ST_parsed"] = raw["ST"].apply(parse_st)
else:
    raw["ST_parsed"] = np.nan

# ソート（時系列）
raw = raw.sort_values(["date", "race_id"], ascending=[True, True]).reset_index(drop=True)

# ===== 基準列の決定 =====
assert MODE in ("entry", "wakuban"), "MODE は 'entry' か 'wakuban' を指定してください。"
COURSE_COL = MODE

# ===== 対象行（race_id, player_id）の基準値を取得 =====
row_cur = raw[(raw["race_id"] == str(TARGET_RACE_ID)) & (raw["player_id"] == str(TARGET_PLAYER_ID))]
if row_cur.empty:
    raise ValueError("指定の race_id / player_id が期間内 raw に見つかりません。START/END_DATE や ID を確認してください。")

course_cur = row_cur[COURSE_COL].iloc[0]
date_cur   = row_cur["date"].iloc[0]
race_cur   = row_cur["race_id"].iloc[0]

if pd.isna(course_cur):
    raise ValueError(f"対象行の {COURSE_COL} が欠損です。raw の列名や内容を確認してください。")

# ===== その選手×(entry or wakuban) の直前N走を抽出 =====
gdf = raw[(raw["player_id"] == str(TARGET_PLAYER_ID)) & (raw[COURSE_COL] == course_cur)].copy()

# 「直前」の定義：date が小さい もしくは 同日で race_id が小さいもの
mask_prev = (gdf["date"] < date_cur) | ((gdf["date"] == date_cur) & (gdf["race_id"] < race_cur))
hist = gdf[mask_prev].sort_values(["date", "race_id"]).tail(N_LAST).copy()

# 表示列
cols_show = [
    "date", "race_id", "player_id", COURSE_COL,
    "rank", "rank_tok", "rank_num", "started_mask",
    "ST", "ST_parsed", "__source_file"
]
present = [c for c in cols_show if c in hist.columns]
out = hist[present].reset_index(drop=True)

print(f"基準={COURSE_COL}, 値={int(course_cur)} / 対象: player_id={TARGET_PLAYER_ID}, race_id={TARGET_RACE_ID}")
print(f"直前{len(out)}走（最大{N_LAST}）")
display(out)


基準=wakuban, 値=4 / 対象: player_id=5217, race_id=202510062412
直前10走（最大10）


,date,race_id,player_id,wakuban,rank,rank_tok,rank_num,started_mask,ST,ST_parsed,__source_file
0,2025-06-30,202506302009,5217,4,１,1,1.0,True,0.09,0.09,20250630_raw.csv
1,2025-07-03,202507032012,5217,4,１,1,1.0,True,0.19,0.19,20250703_raw.csv
2,2025-07-22,202507221007,5217,4,１,1,1.0,True,0.12,0.12,20250722_raw.csv
3,2025-07-26,202507261010,5217,4,５,5,5.0,True,0.16,0.16,20250726_raw.csv
4,2025-07-27,202507271011,5217,4,２,2,2.0,True,0.17,0.17,20250727_raw.csv
5,2025-08-07,202508070412,5217,4,２,2,2.0,True,0.17,0.17,20250807_raw.csv
6,2025-08-10,202508100408,5217,4,２,2,2.0,True,0.08,0.08,20250810_raw.csv
7,2025-08-12,202508120412,5217,4,３,3,3.0,True,0.21,0.21,20250812_raw.csv
8,2025-09-26,202509262102,5217,4,１,1,1.0,True,0.16,0.16,20250926_raw.csv
9,2025-09-29,202509292110,5217,4,３,3,3.0,True,0.11,0.11,20250929_raw.csv
